# SketchGraphs 튜토리얼

이 튜토리얼에서는 SketchGraphs 라이브러리를 사용하여 2D 스케치 데이터를 다루는 방법을 학습합니다.

## 개요
SketchGraphs는 CAD 스케치를 그래프 형태로 표현하고 분석할 수 있는 Python 라이브러리입니다. 이 라이브러리를 통해:
- 스케치 시퀀스 데이터를 로드하고 분석할 수 있습니다
- 스케치를 시각화할 수 있습니다
- 스케치를 그래프 구조로 변환하여 분석할 수 있습니다

## 목차
1. [환경 설정](#1-환경-설정)
2. [라이브러리 임포트](#2-라이브러리-임포트)
3. [데이터 준비](#3-데이터-준비)
4. [데이터 로드 및 탐색](#4-데이터-로드-및-탐색)
5. [스케치 시각화](#5-스케치-시각화)
6. [그래프 표현 및 시각화](#6-그래프-표현-및-시각화)

## 1. 환경 설정

먼저 필요한 패키지들을 설치합니다.

In [ ]:
# SketchGraphs 라이브러리를 개발 모드로 설치
%pip install -e SketchGraphs

In [ ]:
# 데이터 압축 해제를 위한 lz4 패키지 설치
%pip install lz4

### 그래프 시각화를 위한 pygraphviz 설치

운영체제에 따라 다른 설치 방법을 사용합니다:

In [ ]:
# MacOS 사용자용 - Homebrew를 통해 graphviz가 설치되어 있어야 합니다
%pip install \
--global-option=build_ext \
--global-option="-I$(brew --prefix graphviz)/include" \
--global-option="-L$(brew --prefix graphviz)/lib" \
pygraphviz

In [ ]:
# Windows 사용자용 (위의 MacOS 설치가 실패할 경우 이 방법을 사용하세요)
# %pip install pygraphviz

## 2. 라이브러리 임포트

필요한 Python 라이브러리들을 임포트합니다.

In [ ]:
# 시각화 및 수치 계산
from matplotlib import pyplot as plt

# SketchGraphs 라이브러리
import sketchgraphs.data as datalib
from sketchgraphs.data import flat_array

## 3. 데이터 준비

SketchGraphs 데이터셋을 사용하기 위해 검증 데이터셋을 다운로드해야 합니다.

### 데이터 다운로드
- 다운로드 링크: [validation set](https://sketchgraphs.cs.princeton.edu/sequence/sg_t16_validation.npy)
- 다운로드 받은 `sg_t16_validation.npy` 파일을 `SketchGraphs/assets/` 폴더에 저장하세요.

**중요**: 파일을 다운로드한 후 `SketchGraphs/assets/` 디렉토리에 배치해야 다음 단계를 진행할 수 있습니다.

## 4. 데이터 로드 및 탐색

이제 다운로드한 데이터를 로드하고 내용을 살펴보겠습니다.

In [ ]:
# 검증 데이터셋 로드
seq_data = flat_array.load_dictionary_flat('SketchGraphs/assets/sg_t16_validation.npy')

# 데이터 구조 확인
print("데이터 키:", list(seq_data.keys()))
print("시퀀스 개수:", len(seq_data['sequences']))

In [ ]:
# 전체 시퀀스 정보 출력
seq_data['sequences']

이 파일에는 총 315,228개의 시퀀스가 포함되어 있습니다. 이제 그 중 하나의 시퀀스를 선택하여 자세히 살펴보겠습니다.

In [ ]:
# 특정 시퀀스 선택 (인덱스 1327번)
seq = seq_data['sequences'][1327]

# 시퀀스의 처음 20개 연산 출력
print("시퀀스의 처음 20개 연산:")
print(*seq[:20], sep='\n')

### 시퀀스 구조 이해

위에서 볼 수 있듯이, 구성 시퀀스는 다음과 같은 요소들의 리스트입니다:
- **`NodeOp`**: 기본 도형(엔티티)의 추가를 나타냅니다 (예: 선, 원, 점 등)
- **`EdgeOp`**: 제약조건의 추가를 나타냅니다 (예: 평행, 수직, 거리 제약 등)

이러한 연산들이 순차적으로 실행되어 최종 스케치가 완성됩니다.

## 5. 스케치 시각화

이제 시퀀스로부터 스케치 객체를 생성하고 시각화해보겠습니다.

In [ ]:
# 시퀀스로부터 스케치 객체 생성
sketch = datalib.sketch_from_sequence(seq)

# 스케치 렌더링 (일반 모드)
print("일반 스케치 렌더링:")
datalib.render_sketch(sketch);

### 손그림 스타일 렌더링

matplotlib의 xkcd 모드를 사용하여 손그림 스타일로도 렌더링할 수 있습니다.

In [ ]:
# 손그림 스타일로 스케치 렌더링
print("손그림 스타일 스케치 렌더링:")
datalib.render_sketch(sketch, hand_drawn=True);

## 6. 그래프 표현 및 시각화

스케치를 그래프 구조로 변환하여 시각화해보겠습니다. 이를 통해 스케치의 구조적 관계를 더 잘 이해할 수 있습니다.

In [ ]:
# 시퀀스로부터 그래프 표현 생성
G = datalib.pgvgraph_from_sequence(seq)

# 그래프를 이미지 파일로 렌더링
datalib.render_graph(G, '/tmp/my_graph.png')

# 생성된 그래프 이미지 로드 및 표시
img = plt.imread('/tmp/my_graph.png')
fig = plt.figure(dpi=500)

# 그래프가 크므로 일부분만 표시 (중앙 부분)
plt.imshow(img[:, 500:1700])
plt.axis('off')
plt.title('스케치 그래프 구조 (일부분)')
plt.show()

### 그래프 구조 이해

위의 그래프에서 볼 수 있는 요소들:

- **노드**: 스케치의 기본 요소들 (점, 선, 원 등)
- **엣지**: 요소들 간의 관계 (제약조건)
- **`SN`으로 시작하는 라벨**: **서브노드(subnodes)**를 나타냅니다
  - 이는 특정 기본 도형 위의 점을 지정합니다
  - 예: 선분의 끝점, 원의 중심점 등

**참고**: 이 예제의 전체 그래프 이미지는 매우 크기 때문에 위에서는 일부분만 표시했습니다.

## 요약

이 튜토리얼에서 다룬 내용:

1. **환경 설정**: SketchGraphs와 필요한 패키지들 설치
2. **데이터 로드**: 검증 데이터셋을 로드하고 구조 파악
3. **시퀀스 분석**: NodeOp와 EdgeOp로 구성된 스케치 구성 시퀀스 이해
4. **스케치 시각화**: 일반 모드와 손그림 스타일로 스케치 렌더링
5. **그래프 표현**: 스케치를 그래프 구조로 변환하여 구조적 관계 시각화

### 다음 단계

이제 SketchGraphs의 기본 사용법을 익혔으므로, 다음과 같은 고급 기능들을 탐색해볼 수 있습니다:
- 스케치 생성 및 편집
- 제약조건 분석
- 머신러닝 모델을 위한 데이터 전처리
- 자동 스케치 생성
- [sketchgraphs_demo](SketchGraphs/demos/sketchgraphs_demo.ipynb)를 확인하여 onshape을 다루는 부분까지 확인할 수 있습니다.